In [ ]:
yoi = [2020]
toi = ['15RTN', '15RTP', '14RNS']
root_path = "C:/Users/mcecil/CGA/CDL/"
spath = root_path + f"CDL_HLS_dataframe{yoi[0]}.csv"
url_file = root_path + f'CDL_HLS_dataframe{yoi[0]}.csv'
index_file = root_path + f'mask_index_{yoi[0]}.csv'
kml_file = root_path + 'sentinel_tile_grid.kml'
geojson_file = root_path + 'aoi.geojson'  ## chip file, lat-long
geojson_rpj_file = root_path + 'aoi_rpj.geojson'
hdf_dir = root_path + 'hdf/'
tiff_dir = root_path + 'tif/'
mask_dir = root_path + 'mask/'

tile_src_path = root_path + "tile_src.csv"

# New tile edge length (in pixels)
npixels = 512

# Directory containing scenes
scdir = root_path + 'scenes/'

# Directory containing masks
madir = root_path +  'masks/'

# Location to save new files
svdir = root_path +  'subsetted_USSE/'


In [ ]:

# Switch plotting backend
import matplotlib
matplotlib.use('agg')

# Import required modules
#import cartopy.crs as ccrs
import os
from datetime import datetime
import pandas as pd
import geopandas
#import pyhdf
import rasterio
import rasterio.mask
import matplotlib.pyplot as pp
import nasa_hls
import numpy as np
import time
import fiona
from pathlib import Path
import time
import urllib.request as urlreq
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [ ]:
# Switch plotting backend
import matplotlib
matplotlib.use('agg')


### Import modules
from glob import glob
import numpy as np
import matplotlib.pyplot as pp
import rasterio

In [ ]:
from rasterio.windows import Window

### Locate mask files and loop over them
masks = sorted(glob(madir+'*.tif'))
#print(masks)

Adapted code from burn scar workflow (not working)

In [ ]:
from rasterio.windows import Window

# Scale value for HLS S30 bands
scale = 0.0001

### Locate mask files and loop over them
masks = sorted(glob(madir+'*.tif'))

for mask in masks[0:10]:
    print(mask)
    mask = mask.replace('\\', '/')
    # Read data
    src = rasterio.open(mask)
    data = src.read(1)
    
    # Skip if no burn scar
    if (np.sum(data == 1) == 0):
        print('skipped')
        continue
    
    # Locate burn scar center point
    xinds = np.arange(data.shape[0])
    yinds = np.arange(data.shape[1])
    xg, yg = np.meshgrid(xinds, yinds)
    xc = np.floor(np.mean(xg[data==1]))
    yc = np.floor(np.mean(yg[data==1]))
    
    # Compute edges of new box
    x1 = int(xc-npixels//2)
    x2 = int(xc+npixels//2)
    y1 = int(yc-npixels//2)
    y2 = int(yc+npixels//2)
    
    # Check if new inds are out of bounds
    if (x2 > data.shape[0]+1):
        shift = x2-((data.shape[0]+1))+1
        x2 = x2 - shift
        x1 = x1 - shift
        
    if (x1 < 0):
        shift = abs(x1)+2
        x1 = x1 + shift
        x2 = x2 + shift
        
    if (y2 > data.shape[1]+1):
        shift = (y2-(data.shape[1]+1))+1
        y2 = y2 - shift
        y1 = y1 - shift
        
    if (y1 < 0):
        shift = abs(y1)+2
        y1 = y1 + shift
        y2 = y2 + shift

    # Subset the mask
    subset_data = np.expand_dims(data[y1:y2, x1:x2], axis=0)
            
    ### Grab matching scene files
    name = mask.replace('\\', '/').split("/")[-1].replace(".mask.tif", "")
    print(name)

    scene_dir = f'{tiff_dir}{name}'
    print(scene_dir)
   # scene_dir = f'{scdir}'

    ### Comment out unwanted bands
    #src1 = rasterio.open(f'{scene_dir}/{name}__Coastal_Aerosol.tif')
    src2 = rasterio.open(f'{scene_dir}/{name}__Blue.tif')
    src3 = rasterio.open(f'{scene_dir}/{name}__Green.tif')
    src4 = rasterio.open(f'{scene_dir}/{name}__Red.tif')
    #src5 = rasterio.open(f'{scene_dir}/{name}__Red_Edge1.tif')
    #src6 = rasterio.open(f'{scene_dir}/{name}__Red_Edge2.tif')
    #src7 = rasterio.open(f'{scene_dir}/{name}__Red_Edge3.tif')
    #src8 = rasterio.open(f'{scene_dir}/{name}__NIR_Broad.tif')
    src8A = rasterio.open(f'{scene_dir}/{name}__NIR_Narrow.tif')
    src11 = rasterio.open(f'{scene_dir}/{name}__SWIR1.tif')
    src12 = rasterio.open(f'{scene_dir}/{name}__SWIR2.tif')
    #src9 = rasterio.open(f'{scene_dir}/{name}__Water_Vapor.tif')
    #src10 = rasterio.open(f'{scene_dir}/{name}__Cirrus.tif')
    
    bands = []
    for src in (src2,src3,src4,src8A,src11,src12): # Set the tuple to match desired bands
    
        # Scale and clip reflectances
        band = np.clip(src.read(1)[y1:y2, x1:x2]*scale, 0, 1)
        bands.append(band)
        
    bands = np.array(bands)
        
    ### Locate missing data and apply to the mask file
    missing_data = bands[0] == 0
    subset_data[0, missing_data] = -1
    
    # Plot the new mask    
    fig, (ax1, ax2) = pp.subplots(ncols=2)
    ax1.imshow(data)
    ax1.set_title('Whole Scene Mask', fontsize=14, fontweight='roman')
    ax2.imshow(subset_data[0,:,:])
    ax2.set_title('Subsetted', fontsize=14, fontweight='roman')
    pp.savefig(f'{svdir}/mask_plots/{mask.split("/")[-1].replace("tif", "jpg")}')
    pp.close()
        
    # Save the new mask
    out_meta = src.meta
    out_meta.update({'driver':'GTiff', 'height':subset_data.shape[1],
        'width':subset_data.shape[2]})
    with rasterio.open(f'{svdir}/masks/subsetted_{npixels}x{npixels}_{mask.split("/")[-1]}', 'w', **out_meta) as dest:
        dest.write(subset_data)
        
    ### Save the scene tiff
    out_meta.update({'driver':'GTiff', 'height':bands.shape[1],
        'width':bands.shape[2], 'count':bands.shape[0], 'dtype':bands.dtype})
    save_name = f'{svdir}/scenes/subsetted_{npixels}x{npixels}_{name}_merged.tif'
    with rasterio.open(save_name, 'w', **out_meta) as dest:
        dest.write(bands)

Adjusted code using rasterio mask with crop = True (works, needs cleanup)

In [ ]:
# Open the geojson file and extract the bounding box coordinates
with open(geojson_file) as f:
    aoi_data = json.load(f)
    coords = aoi_data['features'][0]['geometry']['coordinates'][0]

aoi_src = geopandas.read_file(geojson_file)
aoi_src.iloc[0].geometry
print(aoi_src.crs)

In [ ]:
from rasterio.windows import Window
import rasterio
from rasterio.mask import mask as m
import json


    
    
    
    
# Scale value for HLS S30 bands
scale = 0.0001



### Locate mask files and loop over them
masks = sorted(glob(madir+'*.tif'))

for mask in masks[0:10]:
    print(mask)
    mask = mask.replace('\\', '/')
    # Read data
    src = rasterio.open(mask)
    mask_image, mask_transform = rasterio.mask.mask(rasterio.open(mask), [aoi_shape.geometry], crop=True)
    src_data = mask_image
    print(src_data)
    print(src_data.shape)
    
    subset_data = src_data
    
   # print(src.shape)
    data = src_data[0] #.read(1)
    
    # Skip if no burn scar
    if (np.sum(data == 1) == 0):
        print('skipped')
        continue
    
    # Locate burn scar center point
#     xinds = np.arange(data.shape[0])
#     print(xinds)
#     yinds = np.arange(data.shape[1])
#     print(yinds)
#     print(xg.shape)
#     print(yg.shape)
#     print(data.shape)
#     xg, yg = np.meshgrid(xinds, yinds)
#     xc = np.floor(np.mean(xg[data==1]))
#     yc = np.floor(np.mean(yg[data==1]))
    
#     # Compute edges of new box
#     x1 = int(xc-npixels//2)
#     x2 = int(xc+npixels//2)
#     y1 = int(yc-npixels//2)
#     y2 = int(yc+npixels//2)
    
#     # Check if new inds are out of bounds
#     if (x2 > data.shape[0]+1):
#         shift = x2-((data.shape[0]+1))+1
#         x2 = x2 - shift
#         x1 = x1 - shift
        
#     if (x1 < 0):
#         shift = abs(x1)+2
#         x1 = x1 + shift
#         x2 = x2 + shift
        
#     if (y2 > data.shape[1]+1):
#         shift = (y2-(data.shape[1]+1))+1
#         y2 = y2 - shift
#         y1 = y1 - shift
        
#     if (y1 < 0):
#         shift = abs(y1)+2
#         y1 = y1 + shift
#         y2 = y2 + shift

    # Subset the mask
  #  subset_data = np.expand_dims(data[y1:y2, x1:x2], axis=0)
            
    ### Grab matching scene files
    name = mask.replace('\\', '/').split("/")[-1].replace(".mask.tif", "")
    print(name)

    scene_dir = f'{tiff_dir}{name}'
    print(scene_dir)
   # scene_dir = f'{scdir}'

    ### Comment out unwanted bands
    #src1 = rasterio.open(f'{scene_dir}/{name}__Coastal_Aerosol.tif')
    src2 = f'{scene_dir}/{name}__Blue.tif'
    src3 = (f'{scene_dir}/{name}__Green.tif')
    src4 = (f'{scene_dir}/{name}__Red.tif')
    #src5 = rasterio.open(f'{scene_dir}/{name}__Red_Edge1.tif')
    #src6 = rasterio.open(f'{scene_dir}/{name}__Red_Edge2.tif')
    #src7 = rasterio.open(f'{scene_dir}/{name}__Red_Edge3.tif')
    #src8 = rasterio.open(f'{scene_dir}/{name}__NIR_Broad.tif')
    src8A = (f'{scene_dir}/{name}__NIR_Narrow.tif')
    src11 = (f'{scene_dir}/{name}__SWIR1.tif')
    src12 = (f'{scene_dir}/{name}__SWIR2.tif')
    #src9 = rasterio.open(f'{scene_dir}/{name}__Water_Vapor.tif')
    #src10 = rasterio.open(f'{scene_dir}/{name}__Cirrus.tif')
    
    
# Convert burn scar shape to proper geometry
    aoi_shape = aoi_src.to_crs(src.crs).iloc[0]

    print(aoi_shape)
        # Build the mask
    #out_image, out_transform = rasterio.mask.mask(src, [aoi_shape.geometry], crop=False)
    
   ## read in blue band for transformation
  #  with rasterio.open(src2) as src:

    out_image_template, out_transform_template = rasterio.mask.mask(rasterio.open(src2), [aoi_shape.geometry], crop=True)
    print(out_image_template.shape)
    with rasterio.open(src2) as src:
        meta = src.meta.copy()
  #  meta = src.meta.copy()
  #  m(src, [aoi_shape.geometry], crop=True)
        # Scale and clip reflectances
        band = np.clip(out_image_template*scale, 0, 1)[0]
       # bands.append(band)
    print(band.shape)
    bands = [band]
    
    for src in (src3,src4,src8A,src11,src12): # Set the tuple to match desired bands
        out_image, out_transform = rasterio.mask.mask(rasterio.open(src), [aoi_shape.geometry], crop=True)
        # Scale and clip reflectances
        band = np.clip(out_image*scale, 0, 1)[0]
        bands.append(band)
        
    bands = np.array(bands)
    print(bands)
    print(bands.shape)
    ### Locate missing data and apply to the mask file
    missing_data = bands[0] == 0
  #  subset_data[0, missing_data] = -1
    
    # Update the metadata with the new dimensions and transform
    meta.update({"driver": "GTiff",
                 "height": out_image_template.shape[1],
                 "width": out_image_template.shape[2],
                 "transform": out_transform_template})

    # Save the cropped image to a new .tif file
   # with rasterio.open("output_cropped.tif", "w", **meta) as dest:
    #    dest.write(out_image)
    
    

    # Plot the new mask    
    fig, (ax1, ax2) = pp.subplots(ncols=2)
    ax1.imshow(data)
    ax1.set_title('Whole Scene Mask', fontsize=14, fontweight='roman')
    ax2.imshow(subset_data[0,:,:])
    ax2.set_title('Subsetted', fontsize=14, fontweight='roman')
    pp.savefig(f'{svdir}/mask_plots/{mask.split("/")[-1].replace(".tif", "_2.jpg")}')
    pp.close()
        
    # Save the new mask
   # out_meta = src.meta.copy()
   # out_meta.update({'driver':'GTiff', 'height':subset_data.shape[1],
   #     'width':subset_data.shape[2]})
    with rasterio.open(f'{svdir}/masks/subsetted_{npixels}x{npixels}_{mask.split("/")[-1].replace(".tif", "_2.tif")}', 'w', **meta) as dest:
        dest.write(subset_data)
    
                 
    meta.update({"count": 6,
                'dtype': 'float32'})
    ### Save the scene tiff
    out_meta.update({'driver':'GTiff', 'height':bands.shape[1],
        'width':bands.shape[2], 'count':bands.shape[0], 'dtype':bands.dtype})
    save_name = f'{svdir}/scenes/subsetted_{npixels}x{npixels}_{name}_merged_2.tif'
    print(meta)
    print(bands.shape)
    with rasterio.open(save_name, 'w', **meta) as dest:
        dest.write(bands)

In [ ]:
from osgeo import gdal

raster_bounds = src.bounds
print(raster_bounds)
upper_left_x = 699934.584491
upper_left_y = 6169364.0093
lower_right_x = 700160.946739
lower_right_y = 6168703.00544
window = (upper_left_x,upper_left_y,lower_right_x,lower_right_y)

gdal.Translate('output_crop_raster.tif', 'input_raster.tif', projWin = window)